In [8]:
import pandas as pd
import yfinance as yf
import time

In [9]:
# Define time intervals for closing prices
time_intervals = {"1h": "60m", "1d": "1d", "1w": "1wk", "1m": "1mo"}

# Load data frame
data = pd.read_csv("/Users/teekyboy/Desktop/random_test.csv")

In [10]:
# Loop through stock symbols
for symbol in data["stock"]:
    # Get stock data from Yahoo Finance
    ticker = yf.Ticker(symbol)
    stock_data = ticker.history(period="max")
    if not stock_data.empty:
        # Get buying price at the time of the date column
        date_str = data.loc[data["stock"]==symbol, "date"].values[0]
        date = pd.to_datetime(date_str)
        price = stock_data.loc[date, "Close"]
        if pd.isna(price):
            # If price is not available, get opening price of next day
            next_date = date + pd.Timedelta(days=1)
            next_price = stock_data.loc[next_date, "Open"]
            if not pd.isna(next_price):
                price = next_price
        # Update price column
        data.loc[data["stock"]==symbol, "price"] = price
        # Get closing prices for different time intervals
        for interval, period in time_intervals.items():
            try:
                closing_data = ticker.history(period=period)
                if not closing_data.empty:
                    closing_price = closing_data["Close"].iloc[-1]
                    data.loc[data["stock"]==symbol, f"closing_{interval}"] = closing_price
            except OSError:
                print(f"Could not get closing price for {interval} for symbol {symbol}")
    else:
        print(f"No data found for symbol {symbol}")
    # Sleep for 1 second to avoid API rate limit
    time.sleep(1)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "pandas/_libs/index.pyx", line 548, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 2263, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 2273, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1640999757000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/teekyboy/.pyenv/versions/3.10.6/envs/twitter_files/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 516, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas/_libs/index.pyx", line 550, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2021-12-31 20:15:57-0500', tz='America/New_York')

The above exception was the direct cause of the following exception:

Traceback (